<a href="https://colab.research.google.com/github/ElisabethShah/DS-Unit-2-Tree-Ensembles/blob/master/module2-random-forests/Random%20Forests%20-%20Assignment%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science — Tree Ensembles_ 

This sprint, your project is about water pumps in Tanzania. Can you predict which water pumps are faulty?

# Random Forests, Ordinal Encoding

## Objectives

* do feature engineering with dates
* clean data with outliers
* impute missing values
* do ordinal encoding with high-cardinality categoricals
* use scikit-learn for decision trees & random forests
* get and interpret feature importances of a tree-based model
* understand how tree ensembles reduce overfitting compared to a single decision tree with unlimited depth
* understand how categorical encodings affect trees differently compared to linear models



## Summary 

**Try Tree Ensembles when you do machine learning with labeled, tabular data**
- "Tree Ensembles" means Random Forest or Gradient Boosting models. 
- [Tree Ensembles often have the best predictive accuracy](https://arxiv.org/abs/1708.05070) with labeled, tabular data.
- Why? Because trees can fit non-linear, non-[monotonic](https://en.wikipedia.org/wiki/Monotonic_function) relationships, and [interactions](https://christophm.github.io/interpretable-ml-book/interaction.html) between features.
- A single decision tree, grown to unlimited depth, will [overfit](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/). We solve this problem by ensembling trees, with bagging (Random Forest) or boosting (Gradient Boosting).
- Random Forest's advantage: may be less sensitive to hyperparameters. Gradient Boosting's advantage: may get better predictive accuracy.

**One-hot encoding isn’t the only way, and may not be the best way, of categorical encoding for tree ensembles.**
- For example, tree ensembles can work with arbitrary "ordinal" encoding! (Randomly assigning an integer to each category.) Compared to one-hot encoding, the dimensionality will be lower, and the predictive accuracy may be just as good or even better.


## Set up environment

In [2]:
# Install dependencies.
!pip install category_encoders

     |████████████████████████████████| 92kB 3.4MB/s 


In [0]:
# Import libraries.
%matplotlib inline

import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.tree import DecisionTreeClassifier

## Load data

In [0]:
# Define data source location.
LOCAL = '../data/tanzania/'
WEB = ('https://raw.githubusercontent.com/LambdaSchool/'
       'DS-Unit-2-Tree-Ensembles/master/data/tanzania/')
source = WEB

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(source + 'train_features.csv'), 
                 pd.read_csv(source + 'train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(source + 'test_features.csv')
sample_submission = pd.read_csv(source + 'sample_submission.csv')

# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=0)